In [14]:
#this is a datascience training project in which I had to create a neural network 
#to predict the kind of trees will grow in an area based on cartographic data

#first I loaded the libraries
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import  Dense, Dropout


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [15]:
# loading the data
data = pd.read_csv("cover_data.csv")

In [16]:
#looking at the data

data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [17]:
#to avoid writing all the features one-by-one, this command gives me a list of columns, 
#I'll just need to remove the last one that gives the tree-cover type, which is what we are training the 
#algorithm to predict

data.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [18]:
#get a list of all the cartographic data into a dataframe, 
#this is going to be our input in the neural network

features = data[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40']]

In [19]:
#create a dataframe with the labels, which is what we want to predict with the algorythm

labels = data["class"]

In [20]:
#look at the shape of the labels, it seems that type 1 and type 2 treecovers are the most abundant

labels.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: class, dtype: int64

In [21]:
#divide the data into training sets and testing sets, like the name indicates,
#like the name indicates, the training set is used to train the neural network,
#and the test set is used to evaluate it's accuracy

x_train, x_test, y_train, y_test = train_test_split(features, labels)

In [22]:
#before we start, we need to make sure that the data is normalized,
#we do that by looking at each feature and normalizing it in a way that 
#it's average becomes 0 and std deviation becomes 1

ct = StandardScaler()
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

In [23]:
#now we are ready to create the neural network, we create chose the kind of model...
model = Sequential()

In [24]:
#we create the imput layer
model.add(tf.keras.layers.InputLayer(input_shape=(54,)))

In [25]:
#and then we build a brain
model.add(Dense(356, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [26]:
#we create a script to train the neural network, with the parameters we are optimizing
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [27]:
#And we input our training data, with 100 epochs the accuracy of the model is 93%.
#You can reduced the number of epochs if you are in a hurry, althought the accuracy won't be as good.
model.fit(x_train, y_train, epochs = 100, batch_size = 5000)

Epoch 1/100
88/88 [==============================] - 4s 42ms/step - loss: 0.8191 - sparse_categorical_accuracy: 0.6807
Epoch 2/100
88/88 [==============================] - 4s 42ms/step - loss: 0.5717 - sparse_categorical_accuracy: 0.7570
Epoch 3/100
88/88 [==============================] - 4s 42ms/step - loss: 0.5163 - sparse_categorical_accuracy: 0.7789
Epoch 4/100
88/88 [==============================] - 4s 46ms/step - loss: 0.4782 - sparse_categorical_accuracy: 0.7963
Epoch 5/100
88/88 [==============================] - 4s 45ms/step - loss: 0.4487 - sparse_categorical_accuracy: 0.8116
Epoch 6/100
88/88 [==============================] - 4s 47ms/step - loss: 0.4284 - sparse_categorical_accuracy: 0.8209
Epoch 7/100
88/88 [==============================] - 4s 43ms/step - loss: 0.4073 - sparse_categorical_accuracy: 0.8310
Epoch 8/100
88/88 [==============================] - 3s 40ms/step - loss: 0.3940 - sparse_categorical_accuracy: 0.8377
Epoch 9/100
88/88 [=============================

88/88 [==============================] - 3s 37ms/step - loss: 0.1936 - sparse_categorical_accuracy: 0.9235
Epoch 70/100
88/88 [==============================] - 4s 44ms/step - loss: 0.1899 - sparse_categorical_accuracy: 0.9249
Epoch 71/100
88/88 [==============================] - 3s 39ms/step - loss: 0.1893 - sparse_categorical_accuracy: 0.9250
Epoch 72/100
88/88 [==============================] - 3s 39ms/step - loss: 0.1989 - sparse_categorical_accuracy: 0.9200
Epoch 73/100
88/88 [==============================] - 4s 40ms/step - loss: 0.1854 - sparse_categorical_accuracy: 0.9264
Epoch 74/100
88/88 [==============================] - 4s 43ms/step - loss: 0.1861 - sparse_categorical_accuracy: 0.9258
Epoch 75/100
88/88 [==============================] - 3s 38ms/step - loss: 0.1866 - sparse_categorical_accuracy: 0.9260
Epoch 76/100
88/88 [==============================] - 4s 45ms/step - loss: 0.1865 - sparse_categorical_accuracy: 0.9260
Epoch 77/100
88/88 [==============================] -

In [28]:
#and we test the model on our validation data

y_estimate = model.predict(x_test)
y_estimate = np.argmax(y_estimate, axis = 1)
print(classification_report(y_test, y_estimate))

4540/4540 [==============================] - 7s 2ms/step
              precision    recall  f1-score   support

           1       0.90      0.95      0.92     52784
           2       0.95      0.92      0.93     70806
           3       0.93      0.90      0.91      9074
           4       0.85      0.85      0.85       697
           5       0.89      0.69      0.78      2390
           6       0.83      0.86      0.84      4397
           7       0.96      0.92      0.94      5105

    accuracy                           0.92    145253
   macro avg       0.90      0.87      0.88    145253
weighted avg       0.93      0.92      0.92    145253

